# This project consit in segmenting and clustering neigbourhood in Toronto

In [2]:

! pip install  bs4

from bs4 import BeautifulSoup


In [3]:
! pip install requests

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt


In [5]:
import requests

## Let scrap the wikipedia data using the ling provided

In [75]:
url=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(url,'lxml')

In [76]:
My_table=soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [8]:
table_data = My_table.tbody.find_all("tr") 

## In the following  I am going to process data Scrapped from wikipedia and transform to a useable data format
**Here we process only data with  Borough assigned and set Neighbourhood to the corresponding Borough when not assigned**

In [83]:
# Data processing
data = {}
data['Postcode']=[]
data['Borough']=[]
data['Neighbourhood']=[]

for i in range(1,len(table_data)):
    Post=table_data[i].find_all("td")[0].text.strip()
    if table_data[i].find_all("a"):
        Bor=table_data[i].find_all("a")[0].get('title')
        if len(table_data[i].find_all("a"))>1:
            Nei=table_data[i].find_all("a")[1].get('title')
        else:
             Nei=Bor
        data['Postcode'].append(Post)
        data['Borough'].append(Bor)
        data['Neighbourhood'].append(Nei)
DataDataFrame =pd.DataFrame.from_dict(data)
DataDataFrame.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park (Toronto)
6,M9A,Queen's Park (Toronto),Queen's Park (Toronto)
7,M1B,"Scarborough, Toronto","Rouge, Toronto"
8,M1B,"Scarborough, Toronto","Malvern, Toronto"
9,M3B,North York,North York


## Let count how many time does each Postcode appear in the data set

In [84]:
counteur=DataDataFrame.Postcode.value_counts()
counteur

M9V    8
M8Y    8
M5V    7
M4V    5
M8Z    5
M9B    5
M6M    4
M1V    4
M9C    4
M9R    4
M1T    3
M2J    3
M5T    3
M3H    3
M1E    3
M1L    3
M8X    3
M8V    3
M1C    3
M6K    3
M1M    3
M6L    3
M1K    3
M5H    3
M5J    3
M1P    3
M5R    3
M2M    2
M1B    2
M6P    2
      ..
M4H    1
M4C    1
M9A    1
M2K    1
M5G    1
M3M    1
M6G    1
M9L    1
M5N    1
M7A    1
M4M    1
M4N    1
M1S    1
M3L    1
M4J    1
M1X    1
M4Y    1
M9N    1
M2H    1
M6C    1
M2N    1
M2R    1
M5C    1
M5E    1
M3A    1
M4E    1
M1W    1
M4R    1
M4A    1
M9P    1
Name: Postcode, Length: 101, dtype: int64

## As they are dupicated zip code we will combine the Neighbourhood corresponding to each PostCode as following

In [85]:
FinalEntryData=DataDataFrame.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
FinalEntryData.head()

,Postcode,Borough,Neighbourhood
0,M1B,"Scarborough, Toronto","Rouge, Toronto,Malvern, Toronto"
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto),Rouge Hill,Port Union..."
2,M1E,"Scarborough, Toronto","Scarborough, Toronto,Morningside, Toronto,West..."
3,M1G,"Scarborough, Toronto","Woburn, Toronto"
4,M1H,"Scarborough, Toronto","Scarborough, Toronto"


In [86]:
FinalEntryData.shape

(101, 3)

# As data has been processed and cleaned,let add latitude and longitude colums


In [91]:

! pip install geocoder

# Being difficult to acces to coordinates unsing geocoder ,we will use the Geospacial_data and use pandas to load


In [92]:
!wget -q -O 'Geospatial_data.csv'  http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [93]:
Geospatial_df = pd.read_csv('Geospatial_data.csv')
Geospatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [94]:

# define the dataframe columns
column_names = ['Postcode','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

import geocoder # import geocoder

for index, row in FinalEntryData.iterrows():
    latitude = Geospatial_df[Geospatial_df["Postal Code"]==row['Postcode']]['Latitude'].values[0]
    longitude = Geospatial_df[Geospatial_df["Postal Code"]==row['Postcode']]['Longitude'].values[0]
    neighborhoods = neighborhoods.append({'Postcode':row['Postcode'],'Borough': row['Borough'],
                                          'Neighborhood': row['Neighbourhood'],
                                          'Latitude': latitude,
                                          'Longitude': longitude}, ignore_index=True)
neighborhoods.head(10)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,"Scarborough, Toronto","Rouge, Toronto,Malvern, Toronto",43.806686,-79.194353
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto),Rouge Hill,Port Union...",43.784535,-79.160497
2,M1E,"Scarborough, Toronto","Scarborough, Toronto,Morningside, Toronto,West...",43.763573,-79.188711
3,M1G,"Scarborough, Toronto","Woburn, Toronto",43.770992,-79.216917
4,M1H,"Scarborough, Toronto","Scarborough, Toronto",43.773136,-79.239476
5,M1J,"Scarborough, Toronto",Scarborough Village,43.744734,-79.239476
6,M1K,"Scarborough, Toronto","Scarborough, Toronto,Ionview,Kennedy Park, Tor...",43.727929,-79.262029
7,M1L,"Scarborough, Toronto","Clairlea,Golden Mile, Toronto,Oakridge, Toronto",43.711112,-79.284577
8,M1M,"Scarborough, Toronto","Cliffcrest,Cliffside, Toronto,Scarborough, Tor...",43.716316,-79.239476
9,M1N,"Scarborough, Toronto","Birch Cliff,Scarborough, Toronto",43.692657,-79.264848
